A notebook analyzing all the ITL sensors in the etraveler. A scoring algorithm is developed and applied at the bottom.

This should be considered as a {\bf pre-filter} only, as it does not take into account other important characteristics, including bad segments, glow, and other problems. The ranking is therefore just the starting point for sensor selection.

It appears there is a problem with how the noise is being pulled out of etraveler, so we'll keep it out of the metric for now and just check the top performers by hand.

In [1]:
# this is the partner code that can run anywhere using the pickled data, produced by script at SLAC
import matplotlib.pyplot as plt
#from matplotlib.backends.backend_pdf import PdfPages
import datetime
import collections
import numpy as np
import argparse
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import scipy as sc
import scipy.signal
import textwrap
import pylab

# this magic command makes plots appear within the notebook
%matplotlib inline

# define figure size parameters to make figures larger than default
figwidth=30
figheight=10


In [2]:
import pickle
# I ran rank_ITLnew.py at SLAC, then pickled the data. 
f=open('ITLnewdump.p','rb')
expDict=pickle.load(f)
ccd_list=pickle.load(f)
bias_dic=pickle.load(f, encoding='latin1')
RTM_dic=pickle.load(f)
f.close()
len(ccd_list), len(bias_dic)

(137, 137)

In [3]:
def get_CTI(d, ccd):

        cti_low_serial = []
        cti_high_serial = []
        cti_low_parallel = []
        cti_high_parallel = []

        for amp in d['cte'][ccd]['steps']['cte_offline']['cte'][1:]:
            cti_low_serial.append(amp['cti_low_serial'])
            cti_high_serial.append(amp['cti_high_serial'])
            cti_low_parallel.append(amp['cti_low_parallel'])
            cti_high_parallel.append(amp['cti_high_parallel'])

        ctils=np.array(cti_low_serial)
        ctihs=np.array(cti_high_serial)
        ctilp=np.array(cti_low_parallel)
        ctihp=np.array(cti_high_parallel)

        
        print(ccd,'\n sum serial low: %8.2e'% ctils.sum(), 
               '\n sum serial high: %8.2e'% ctihs.sum(), 
               '\n worst serial low: %8.2e'% max(cti_low_serial), 
               '\n worst serial high: %8.2e'% max(cti_high_serial),
               '\n worst parallel low: %8.2e'% max(cti_low_parallel), 
               '\n worst parallel high:%8.2e'% max(cti_high_parallel), '\n', '\n')
        

        return np.array(cti_low_serial), np.array(cti_high_serial), np.array(cti_low_parallel), np.array(cti_high_parallel)

In [4]:
def get_readnoise(d, ccd):

        read_noise = []

        for amp in d['readnoise'][ccd]['steps']['read_noise_offline']['read_noise'][1:]:
            read_noise.append(amp['read_noise'])

        return np.array(read_noise)

In [5]:
def get_nonlinearity(d, ccd):

        nonlinearity = []

        for amp in d['nonlinearity'][ccd]['steps']['flat_pairs_offline']['flat_pairs'][1:]:
            nonlinearity.append(amp['max_frac_dev'])

        return np.array(nonlinearity)

In [6]:
def defectsFraction(d, ccd, col_len=2000, totPixels=1024000.0):
        #sum defect types and express as fraction (per segment)
        # totPixels = 1025024 E2V, 1024000 ITL

        amps = []
        numBrightPixels = []
        numBrightColumns = []
        numDarkPixels = []
        numDarkColumns = []
        numTraps = []

        for amp in d['brightdefects'][ccd]['steps']['bright_defects_offline']['bright_defects']:
            numBrightPixels.append(amp['bright_pixels'])
            numBrightColumns.append(amp['bright_columns'])

        for amp in d['darkdefects'][ccd]['steps']['dark_defects_offline']['dark_defects']:
            numDarkPixels.append(amp['dark_pixels'])
            numDarkColumns.append(amp['dark_columns'])

        for amp in d['traps'][ccd]['steps']['traps_offline']['traps']:
            numTraps.append(amp['num_traps'])

        numBrightPixels = np.array(numBrightPixels[1:])
        numBrightColumns = np.array(numBrightColumns[1:])
        numDarkPixels = np.array(numDarkPixels[1:])
        numDarkColumns = np.array(numDarkColumns[1:])
        numTraps = np.array(numTraps[1:])

        return ((numBrightPixels+numDarkPixels+numTraps+col_len*(numBrightColumns + numDarkColumns))/totPixels), numBrightColumns

In [ ]:
def metric(ccd_list, expDict, bias_dic, RTM_dic):
        ccd_dic={}
        for ccd in ccd_list:
            defectsfrac, nbcs = defectsFraction(expDict,ccd)
            nbc=nbcs.sum()

            ctils, ctihs, ctilp, ctihp = get_CTI(expDict,ccd)
            ctemax=max(ctils.max(),ctihs.max())
            rn = get_readnoise(expDict,ccd)

            bias = bias_dic[ccd]
            RTM = RTM_dic[ccd]
            nonlinearity = get_nonlinearity(expDict, ccd)

            # Score = sCTI/2 + defects/0.2% + nonlinearity/1% + Bias/5000
            Score = ctemax/2.e-6 + defectsfrac.mean()/0.002 + max(nonlinearity)/.01 + bias/5000
            #Scorenew = (ctils.sum()+ctihs.sum())/20.e-6 + defectsfrac.mean()/0.002 + nonlinearity.mean()/.01 + bias/5000
            ctterm= (ctils.sum()+ctihs.sum())/40.e-6 + (ctils.max()+ctihs.max())/20.e-6
            defterm= defectsfrac.mean()/0.002
            nlterm=nonlinearity.mean()/.01
            bterm=bias/2000

# there appears to be a problem with how the noise is gathered...looks as if Richard
# is getting the system noise, not the read noise, so we'll leave this off for now,
# and we'll have to check by hand.

            noiseterm=0.
            #print(rn,'\n')
            for noise in np.nditer(rn):
                if (noise>9.):
                    noiseterm+=(noise-9.)/16.
            #print(noiseterm)
            # let's have a look at the noise
            plt.figure()
            plt.plot(rn, 'r*')
            plt.ylim(0,15)
            lab='read noise '+ccd
            plt.suptitle(lab)
            lim9=rn*0.+9.
            plt.plot(lim9, '-r')
            
            Scorenew=ctterm+defterm+nlterm+bterm
        
            ccd_dic[ccd] = [Score,Scorenew,ctterm,defterm,nlterm,bterm, RTM]
            df = pd.DataFrame(ccd_dic).transpose()
            df.columns = ['OldScore', 'Score', 'CtiScore', 'DefectScore','NonlinScore','BiasScore', 'RTM']
            df = df.astype(dtype= {"OldScore":"float64",
                    "Score":"float64","CtiScore":"float64", "DefectScore":"float64","NonlinScore":"float64",
                                   "BiasScore":"float64","RTM":"object"})


        return df

In [ ]:
df =metric(ccd_list, expDict, bias_dic, RTM_dic)
df.RTM=df['RTM'].str[-3:]
df.RTM = pd.to_numeric(df.RTM)
#print (df['RTM'])
#print(df.sort_values(by='Score'))
#df.str.len(df['RTM'])
#x= df['RTM']
#print(x)
#len(x)


ITL-3800C-411 
 sum serial low: 2.79e-05 
 sum serial high: 1.30e-05 
 worst serial low: 2.95e-06 
 worst serial high: 9.98e-07 
 worst parallel low: 4.95e-07 
 worst parallel high:6.49e-07 
 

ITL-3800C-509 
 sum serial low: 4.33e-05 
 sum serial high: 3.28e-05 
 worst serial low: 4.79e-06 
 worst serial high: 4.66e-06 
 worst parallel low: 3.22e-07 
 worst parallel high:4.27e-08 
 

ITL-3800C-262 
 sum serial low: 4.68e-05 
 sum serial high: 4.54e-05 
 worst serial low: 9.16e-06 
 worst serial high: 9.04e-06 
 worst parallel low: 2.81e-07 
 worst parallel high:2.26e-07 
 

ITL-3800C-161 
 sum serial low: 1.04e-05 
 sum serial high: 5.15e-05 
 worst serial low: 2.85e-06 
 worst serial high: 1.65e-05 
 worst parallel low: 4.24e-05 
 worst parallel high:4.09e-06 
 

ITL-3800C-502 
 sum serial low: 1.26e-05 
 sum serial high: 1.47e-05 
 worst serial low: 3.00e-06 
 worst serial high: 1.19e-06 
 worst parallel low: 1.73e-07 
 worst parallel high:4.21e-08 
 

ITL-3800C-466 
 sum serial low

/Users/ritz/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


 
 worst parallel low: 1.97e-07 
 worst parallel high:6.21e-08 
 

ITL-3800C-510 
 sum serial low: 2.27e-05 
 sum serial high: 1.63e-05 
 worst serial low: 3.11e-06 
 worst serial high: 1.21e-06 
 worst parallel low: 1.32e-07 
 worst parallel high:4.27e-08 
 

ITL-3800C-215 
 sum serial low: 5.69e-05 
 sum serial high: 5.90e-05 
 worst serial low: 7.48e-06 
 worst serial high: 8.34e-06 
 worst parallel low: 1.44e-07 
 worst parallel high:7.63e-08 
 

ITL-3800C-452 
 sum serial low: 6.26e-05 
 sum serial high: 5.67e-05 
 worst serial low: 8.51e-06 
 worst serial high: 8.17e-06 
 worst parallel low: 8.65e-07 
 worst parallel high:9.69e-07 
 

ITL-3800C-451 
 sum serial low: 2.02e-05 
 sum serial high: 1.43e-05 
 worst serial low: 1.22e-05 
 worst serial high: 1.13e-06 
 worst parallel low: 2.29e-07 
 worst parallel high:3.63e-08 
 

ITL-3800C-450 
 sum serial low: 5.59e-05 
 sum serial high: 5.30e-05 
 worst serial low: 6.63e-06 
 worst serial high: 6.72e-06 
 worst parallel low: 2.92e-0

ITL-3800C-235 
 sum serial low: 5.23e-05 
 sum serial high: 5.03e-05 
 worst serial low: 7.53e-06 
 worst serial high: 7.42e-06 
 worst parallel low: 1.94e-07 
 worst parallel high:4.85e-08 
 

ITL-3800C-332 
 sum serial low: 3.65e-05 
 sum serial high: 2.33e-05 
 worst serial low: 5.06e-06 
 worst serial high: 2.82e-06 
 worst parallel low: 3.70e-07 
 worst parallel high:6.75e-08 
 

ITL-3800C-333 
 sum serial low: 3.74e-05 
 sum serial high: 1.74e-05 
 worst serial low: 7.02e-06 
 worst serial high: 2.05e-06 
 worst parallel low: 1.52e-07 
 worst parallel high:6.24e-08 
 

ITL-3800C-330 
 sum serial low: 1.50e-04 
 sum serial high: 1.54e-04 
 worst serial low: 1.22e-05 
 worst serial high: 1.27e-05 
 worst parallel low: 8.14e-08 
 worst parallel high:1.20e-08 
 

ITL-3800C-331 
 sum serial low: 7.40e-05 
 sum serial high: 7.65e-05 
 worst serial low: 7.21e-06 
 worst serial high: 7.39e-06 
 worst parallel low: 2.00e-07 
 worst parallel high:8.14e-08 
 

ITL-3800C-336 
 sum serial low


ITL-3800C-248 
 sum serial low: 3.61e-05 
 sum serial high: 3.41e-05 
 worst serial low: 7.06e-06 
 worst serial high: 6.95e-06 
 worst parallel low: 9.28e-08 
 worst parallel high:5.77e-08 
 

ITL-3800C-088 
 sum serial low: 3.03e-05 
 sum serial high: 1.81e-05 
 worst serial low: 6.38e-06 
 worst serial high: 3.03e-06 
 worst parallel low: 3.03e-07 
 worst parallel high:4.21e-08 
 

ITL-3800C-405 
 sum serial low: 1.67e-04 
 sum serial high: 8.43e-05 
 worst serial low: 6.55e-05 
 worst serial high: 1.07e-05 
 worst parallel low: 1.69e-07 
 worst parallel high:4.76e-06 
 

ITL-3800C-406 
 sum serial low: 3.02e-05 
 sum serial high: 1.32e-05 
 worst serial low: 3.19e-06 
 worst serial high: 9.65e-07 
 worst parallel low: 4.92e-07 
 worst parallel high:8.14e-07 
 

ITL-3800C-407 
 sum serial low: 3.10e-05 
 sum serial high: 2.62e-05 
 worst serial low: 4.62e-06 
 worst serial high: 4.59e-06 
 worst parallel low: 4.60e-07 
 worst parallel high:6.44e-08 
 

ITL-3800C-242 
 sum serial lo

In [ ]:
df.plot.scatter(x='Score',y='OldScore')
df.plot.scatter(x='Score',y='BiasScore')
z=df[(df.Score <= 20 ) & (np.isnan(df.RTM))] # select ones not shown to be on rafts in etraveler
#print (z['RTM'])
z.plot.scatter(x='Score',y='OldScore')
z.plot.scatter(x='Score',y='BiasScore')
z.plot.scatter(x='CtiScore',y='BiasScore')
z.hist(column='Score')
#plt.scatter(x, y, 80,'b','^',label='', alpha=0.65)
#print (z['Score'])
#print(z)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
#print(z.sort_values(by='Score'))
z1=z[['Score','BiasScore','CtiScore', 'DefectScore', 'NonlinScore']]
pd.set_option('display.max_rows',200)
print ('number of sensors found in etraveler: ',len(df),'\n')
print ('\n number of unassigned sensors: ',len(z1),'\n')
#print (round(z1.nsmallest(200,'Score'),2))
round(z1.nsmallest(300,'Score'),2)

and we can easily export any of this to an excel file, if you want it.

In [ ]:
df.describe()


In [ ]:
z1.describe()

In [ ]:
# we see that some sensors may be missing from the etraveler search. 
# to help debug, here is the list of sensors found in etraveler, sorted by 
# sensor number:
#list(df.index)